## Gold Fact Table

Creates a Gold fact table gold_fact_team_performance that records team-level performance metrics per game, including goals scored, opponent goals, shots, penalty minutes, blocked shots, saves, and win outcomes.

It aggregates and joins data from multiple Silver tables (silver_game_team_stats, silver_goalie_stats, and silver_game), generating a composite key team_season_id for relational integrity and downstream use.

In [ ]:
-- Step 1: Define the table structure explicitly

DROP TABLE IF EXISTS gold_fact_team_performance;

CREATE TABLE gold_fact_team_performance (
    game_id STRING NOT NULL,
    season_id STRING NOT NULL,
    team_id STRING NOT NULL,
    team_season_id STRING NOT NULL,
    team_goals INT,
    opponent_goals INT,
    won BOOLEAN,
    shots INT,
    penalty_minutes INT,
    blocked INT,
    saves INT
)
USING delta
TBLPROPERTIES (
    'delta.constraints.primaryKey' = 'team_season_id',
    'lineage.foreignKey.game_id'   = 'gold_dim_game.game_id',
    'lineage.foreignKey.team_id'   = 'gold_dim_team.team_id',
    'lineage.foreignKey.season_id' = 'gold_dim_season.season_id',
    'lineage.foreignKey.team_season_id' = 'gold_bridge_team_season_performance.team_season_id'
);

-- Step 2: Insert data into the table
INSERT INTO Gold_Fact_Team_Performance
WITH team_saves AS (
    SELECT
        game_id,
        team_id,
        SUM(saves) AS total_saves
    FROM silver_goalie_stats
    GROUP BY game_id, team_id
),
team_stats AS (
    SELECT 
        a.game_id,
        a.team_id,
        a.won,
        a.shots,
        a.blocked,
        a.pim,
        a.goals AS team_goals,
        b.goals AS opponent_goals
    FROM silver_game_team_stats a
    JOIN silver_game_team_stats b
        ON a.game_id = b.game_id AND a.team_id <> b.team_id
)
SELECT
    game.game_id,
    game.season AS season_id,
    team.team_id,
    CONCAT(game.season,"_",team.team_id) AS team_season_id,
    team.team_goals,
    team.opponent_goals,
    team.won,
    team.shots,
    team.pim AS penalty_minutes,
    team.blocked,
    saves.total_saves AS saves
FROM team_stats team
LEFT JOIN silver_game game 
    ON team.game_id = game.game_id
LEFT JOIN team_saves saves 
    ON game.game_id = saves.game_id 
   AND team.team_id = saves.team_id;

## Gold Dimension Tables

### gold_dim_game

Creates a Gold dimension table gold_dim_game that stores basic game metadata, including game ID, type (e.g. regular, playoff), and game date.

It loads clean data from the silver_game table, converting the game timestamp to a date, and enforces game_id as the primary key to support consistent joins in the semantic model.

In [ ]:
DROP TABLE IF EXISTS gold_dim_game;

CREATE TABLE gold_dim_game (
    game_id STRING NOT NULL,
    game_type STRING,
    game_date Date
)
USING DELTA
TBLPROPERTIES (
    'delta.constraints.game_id.primaryKey' = 'true'
);

In [ ]:
INSERT OVERWRITE gold_dim_game
SELECT
    game_id,
    type AS game_type,
    TO_DATE(date_time_GMT) AS game_date
FROM silver_game;

### gold_dim_team

Creates a Gold dimension table gold_dim_team that stores team reference data, including team ID, full name, short name, and abbreviation.

It extracts and renames fields from the silver_team_info table, and defines team_id as the primary key to support clean joins with fact and bridge tables.

In [ ]:
DROP TABLE IF EXISTS gold_dim_team;

CREATE TABLE gold_dim_team (
    team_id STRING NOT NULL,
    team_name STRING,
    short_name STRING,
    abbreviation STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.constraints.team_id.primaryKey' = 'true'
);

In [ ]:
INSERT OVERWRITE gold_dim_team
SELECT
    team_id,
    teamName AS team_name,
    shortName AS short_name,
    abbreviation
FROM silver_team_info;

### gold_dim_season

Creates a Gold dimension table gold_dim_season to store season-level metadata, including a unique season_id and the corresponding start and end years.

It extracts distinct seasons from the silver_game table and parses the season string to integer year values. The season_id is set as the primary key for reliable data joins.

In [ ]:
DROP TABLE IF EXISTS gold_dim_season;

CREATE TABLE gold_dim_season (
    season_id STRING NOT NULL,
    season_start_year INT,
    season_end_year INT
)
USING DELTA
TBLPROPERTIES (
    'delta.constraints.season_id.primaryKey' = 'true'
);

In [ ]:
INSERT OVERWRITE gold_dim_season
SELECT
    DISTINCT(season) AS season_id,
    CAST(SUBSTRING(CAST(season AS STRING), 1, 4) AS INTEGER) AS season_start_year,
    CAST(SUBSTRING(CAST(season AS STRING), 5, 4) AS INTEGER) AS season_end_year
FROM silver_game;

### gold_dim_player_age

Creates a Gold dimension table gold_dim_player_age that captures player demographic and age details per season and team.

It combines player info with game participation data from Silver tables, calculates player age at the start of each season, and enforces player_id as the primary key. Foreign keys link to season and team dimension tables for referential integrity.

In [ ]:
DROP TABLE IF EXISTS gold_dim_player_age;

CREATE TABLE gold_dim_player_age (
    player_id STRING NOT NULL,
    season_id STRING NOT NULL,
    team_id STRING NOT NULL,
    player_name STRING,
    birth_date DATE,
    age INT
)
USING DELTA
TBLPROPERTIES (
    'delta.constraints.player_id.primaryKey' = 'true',
    'lineage.foreignKey.season_id'   = 'gold_dim_season.season_id',
    'lineage.foreignKey.team_id'   = 'gold_dim_team.team_id'
);

In [ ]:
INSERT OVERWRITE gold_dim_player_age
SELECT DISTINCT
    p.player_id,
    g.season AS season_id,
    stats.team_id,
    CONCAT(p.firstName, ' ', p.lastName) AS player_name,
    p.birthDate AS birth_date,
    FLOOR(DATEDIFF(
        TO_DATE(CONCAT(s.season_start_year, '-10-01'), 'yyyy-MM-dd'),
        p.birthDate
    ) / 365.25) AS age
FROM silver_player_info p
JOIN (
    SELECT player_id, team_id, game_id FROM silver_goalie_stats
    UNION
    SELECT player_id, team_id, game_id FROM silver_game_skater_stats
) stats
  ON p.player_id = stats.player_id
JOIN silver_game g
  ON stats.game_id = g.game_id
JOIN gold_dim_season s
  ON g.season = s.season_id
WHERE p.birthDate IS NOT NULL
  AND s.season_start_year IS NOT NULL;


### gold_dim_archetype

Creates a Gold dimension table gold_dim_archetype that defines team archetypes with descriptive names, definitions, and scoring rules.

It explicitly lists archetype IDs and their characteristics—Attackers, Defenders, Strategists, and Balancers—providing context for how teams are classified based on performance metrics.

In [ ]:
DROP TABLE IF EXISTS gold_dim_archetype;

CREATE TABLE gold_dim_archetype (
    archetype_id INT NOT NULL,
    archetype_name STRING NOT NULL,
    definition STRING,
    metric_rule STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.constraints.archetype_id.primaryKey' = 'true'
);


In [ ]:
INSERT INTO gold_dim_archetype
VALUES
(1, 'Attackers',
 'Offense-driven teams that generate high shot volumes and take risks, often incurring more penalties as a result.',
 'Scored higher when teams take more shots and penalties, reflecting aggressive offensive play.'),

(2, 'Defenders',
 'Defense-focused teams that rely on strong saves and blocked shots to limit opponent scoring.',
 'Scored higher when teams record more saves and blocked shots while conceding fewer goals.'),

(3, 'Strategists',
 'Efficient, disciplined teams that convert opportunities effectively and maintain strong win rates.',
 'Scored higher when teams have strong shot efficiency, win frequently, and take fewer penalties.'),

(4, 'Balancers',
 'Teams that maintain steady, mid-range performance across metrics without distinct extremes.',
 'Scored higher when teams stay close to the league average across penalties, saves, and efficiency.');

### gold_dim_date

Creates a Gold dimension table gold_dim_date that captures detailed date attributes for each game date, including year, month, day, day of week, season, and game type (Regular, Playoffs, Preseason).

The table is populated from distinct game dates in the silver_game table, with a formatted string date_id for easy reference and foreign key linkage to the season dimension.

In [ ]:
-- Step 1: Drop/Create the date dimension table
DROP TABLE IF EXISTS gold_dim_date;

CREATE TABLE gold_dim_date (
    date_id STRING NOT NULL,     -- e.g. '20251003'
    year INT NOT NULL,
    month INT NOT NULL,
    day_of_month INT NOT NULL,
    day_of_week STRING NOT NULL, -- e.g. 'Monday'
    season_id STRING NOT NULL,   -- from silver_game.season
    game_type STRING             -- Regular / Playoffs / Preseason
)
USING delta
TBLPROPERTIES (
    'lineage.foreignKey.season_id' = 'dim_game_season.season_id'
);

-- Step 2: Insert distinct dates from silver_game
INSERT INTO gold_dim_date
SELECT DISTINCT
    DATE_FORMAT(date_time_GMT, 'yyyyMMdd') AS date_id,
    YEAR(date_time_GMT) AS year,
    MONTH(date_time_GMT) AS month,
    DAY(date_time_GMT) AS day_of_month,
    DATE_FORMAT(date_time_GMT, 'EEEE') AS day_of_week,
    season AS season_id,
    CASE type
        WHEN 'R' THEN 'Regular'
        WHEN 'P' THEN 'Playoffs'
        WHEN 'PR' THEN 'Preseason'
        ELSE type
    END AS game_type
FROM silver_game;


## GOLD Bridge/Aggregate Table

### gold_bridge_team_season_performance

Creates a table that summarizes team performance metrics per season — including game count, average team and opponent goals, shots, penalty minutes, saves, blocked shots, wins, win percentage, efficiency, and penalty rate.

It aggregates data from the gold_fact_team_performance table, grouped by season and team, and generates a unique team_season_id as the primary key for each record.

In [ ]:
-- Step 1: Create team-season performance summary table
DROP TABLE IF EXISTS gold_bridge_team_season_performance;

CREATE TABLE gold_bridge_team_season_performance (
    team_season_id STRING NOT NULL,
    season_id STRING NOT NULL,
    team_id STRING NOT NULL,
    game_count INT,
    avg_team_goals FLOAT,
    avg_opponent_goals FLOAT,
    avg_shots FLOAT,
    total_penalty_minutes INT,
    avg_saves FLOAT,
    avg_blocked FLOAT,
    win_count INT,
    win_percentage FLOAT,
    efficiency FLOAT,
    penalty_rate FLOAT,
    archetype_id INT
)
USING delta
TBLPROPERTIES (
    'delta.constraints.primaryKey' = 'team_season_id',
    'lineage.foreignKey.archetype_id' = 'dim_archetype.archetype_id'
);

INSERT INTO gold_bridge_team_season_performance
SELECT
    CONCAT(season_id, '_', team_id) AS team_season_id,
    season_id,
    team_id,
    COUNT(game_id) AS game_count,
    AVG(team_goals) AS avg_team_goals,
    AVG(opponent_goals) AS avg_opponent_goals,
    AVG(shots) AS avg_shots,
    SUM(penalty_minutes) AS total_penalty_minutes,
    AVG(saves) AS avg_saves,
    AVG(blocked) AS avg_blocked,
    SUM(CASE WHEN won THEN 1 ELSE 0 END) AS win_count,
    SUM(CASE WHEN won THEN 1 ELSE 0 END) * 1.0 / COUNT(game_id) AS win_percentage,
    CASE WHEN SUM(shots) = 0 THEN NULL ELSE SUM(team_goals) * 1.0 / SUM(shots) END AS efficiency,
    SUM(penalty_minutes) * 1.0 / COUNT(game_id) AS penalty_rate,
    NULL AS archetype_id
FROM gold_fact_team_performance
GROUP BY season_id, team_id;


In [ ]:
-- Step 2a: Compute continuous scores per season
WITH continuous_scores AS (
    SELECT
        t.team_season_id,  
        t.season_id,
        t.team_id,
        
        -- Percentile ranks (0-1)
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY avg_shots) AS shots_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY penalty_rate) AS penalty_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY avg_opponent_goals ASC) AS opp_goals_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY avg_blocked) AS blocked_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY avg_saves) AS saves_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY efficiency) AS eff_rank,
        PERCENT_RANK() OVER (PARTITION BY season_id ORDER BY win_percentage) AS win_rank
    FROM gold_bridge_team_season_performance t
),

-- Step 2b: Compute weighted archetype scores (rebalanced)
archetype_scores AS (
    SELECT
        season_id,
        team_id,
        team_season_id,

        -- Attackers: emphasize shots and penalties (risk-taking)
        (2 * shots_rank + 1.5 * penalty_rank) AS attacker_score,

        -- Defenders: emphasize saves & blocked, reduce opponent goals slightly
        (2 * saves_rank + 1 * blocked_rank + 0.8 * (1 - opp_goals_rank)) AS defender_score,

        -- Strategists: emphasize efficiency, reduce penalty impact
        (1.5 * eff_rank + 0.8 * (1 - penalty_rank)) AS strategist_score,

        -- Balancers: reward mid-performing, well-rounded teams
        (1 / (1 + ABS(penalty_rank - 0.5))
         + 1 / (1 + ABS(saves_rank - 0.5))
         + 1 / (1 + ABS(eff_rank - 0.5))) AS balancer_score
    FROM continuous_scores
),

-- Step 2c: Assign final archetype based on highest weighted score
final_archetype AS (
    SELECT
        season_id,
        team_id,
        team_season_id,
        CASE
            WHEN GREATEST(attacker_score, defender_score, strategist_score, balancer_score) = attacker_score THEN 1
            WHEN GREATEST(attacker_score, defender_score, strategist_score, balancer_score) = defender_score THEN 2
            WHEN GREATEST(attacker_score, defender_score, strategist_score, balancer_score) = strategist_score THEN 3
            WHEN GREATEST(attacker_score, defender_score, strategist_score, balancer_score) = balancer_score THEN 4
        END AS archetype_id
    FROM archetype_scores
)

-- Step 2d: Update table with precomputed archetype (using composite key)
MERGE INTO gold_bridge_team_season_performance t
USING final_archetype f
ON t.season_id = f.season_id AND t.team_id = f.team_id
-- Alternatively: if you already have team_season_id in the table
-- ON t.team_season_id = f.team_season_id
WHEN MATCHED THEN UPDATE SET
    t.archetype_id = f.archetype_id;

-- Step 2e: Show updated archetype breakdown
SELECT
    CAST(archetype_id AS STRING) AS archetype_label,
    COUNT(*) AS total_teams,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) AS pct_of_total
FROM gold_bridge_team_season_performance
GROUP BY archetype_label
ORDER BY archetype_label;
